---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [133]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [134]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [135]:
import pandas as pd

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
     
    data = open("university_towns.txt",'r')
    rows = [row for row in data]
    region_name = []
    
    current_state = ""
    for index,row in enumerate(rows):
        if "[edit]" in row:
            current_state=row
        else:
            region_name.append([current_state.split("[")[0].strip(),row.split("(")[0].strip()])
    
    df=pd.DataFrame(region_name).rename(columns={0:"State",1:"RegionName"})
    return df

get_list_of_university_towns().shape

(517, 2)

In [136]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    quarterly = pd.ExcelFile("gdplev.xls").parse('Sheet1',skiprows=219,usecols=[4,6]).as_matrix().tolist()
    
    recession,recession_start_index=0,0
    last_gdp=quarterly[0][1]
    for index,quarter in enumerate(quarterly):
        if quarter[1] < last_gdp:
            recession=recession+1
        elif recession < 2:
                recession = 0
                recession_start_index = index + 1
                
        if recession >= 2:
                return quarterly[recession_start_index][0]
                
        last_gdp = quarter[1]
        
    return "No recession"

get_recession_start()

'2008q3'

In [137]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''

    quarterly = pd.ExcelFile("gdplev.xls").parse('Sheet1',skiprows=219,usecols=[4,6]).as_matrix().tolist()
    
    recession,recession_end_index=0,0
    recession_active=False
    last_gdp=quarterly[0][1]
    for index,quarter in enumerate(quarterly):
        if recession_active == False:
            if quarter[1] < last_gdp:
                recession = recession + 1
            elif recession < 2:
                    recession = 0
            if recession >= 2:
                    recession=0
                    recession_active=True
        else:
            if quarter[1] > last_gdp:
                recession = recession + 1
            elif recession < 2:
                recession = 0
            if recession >= 2:
                return quarter[0]

        last_gdp = quarter[1]
        
    return "No recession"

get_recession_end()

'2009q4'

In [138]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    quarterly = pd.ExcelFile("gdplev.xls").parse('Sheet1',skiprows=219,usecols=[4,6]).as_matrix().tolist()
    
    recession,recession_end_index=0,0
    recession_active=False
    recession_bottom=0
    last_gdp=quarterly[0][1]
    for index,quarter in enumerate(quarterly):
        if recession_active == False:
            if quarter[1] < last_gdp:
                recession = recession + 1
            elif recession < 2:
                    recession = 0
            if recession >= 2:
                    recession=0
                    recession_bottom=index
                    recession_active=True
        else:
            if quarter[1] > last_gdp:
                recession = recession + 1
            else:
                if quarterly[recession_bottom][1] > quarterly[index][1]:
                    recession_bottom=index
                if recession < 2:
                    recession = 0
                
            if recession >= 2:
                return quarterly[recession_bottom][0]

        last_gdp = quarter[1]
        
    return "No recession"

get_recession_bottom()

'2009q2'

In [189]:
def convert_abbrev(a_state,get_mode=True):
    if get_mode:
        for abbrev, state in states.items():
            if a_state == state:
                    return abbrev
    else:
        for abbrev, state in states.items():
            if a_state == abbrev:
                    return state
    return "Not Found"

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    cols=[1,2]+[i for i in range(51,251)]
    df = pd.read_csv("City_Zhvi_AllHomes.csv",usecols=cols).convert_objects(convert_numeric=True)
    df["State"]=df["State"].apply(lambda abbrev:convert_abbrev(abbrev,False))
    df=df.set_index(["State","RegionName"])
    quarters=[]
    for i in range(2000,2016):
        for j in range(1,5):
            quarters.append(str(i)+"q"+str(j))
            quarters.append(str(i)+"q"+str(j))
            quarters.append(str(i)+"q"+str(j))
    for j in range(1,3):
        quarters.append("2016q0"+str(j))
        quarters.append("2016q0"+str(j))
        quarters.append("2016q0"+str(j))
    
    quarters.append("2016q03")
    quarters.append("2016q03")
#     print(quarters)
    df.columns=quarters
    df=df.mean(axis=1, level=0)
    
    return df

convert_housing_data_to_quarters().shape

/Users/soodd/anaconda/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:24: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


(10730, 67)

In [188]:
import scipy.stats as stats
    
def one_quarter_before(quarter):
    cur_year=int(quarter[0:4])
    cur_quarter=int(quarter[5:])
    prev_year=cur_year
    prev_quarter=cur_quarter-1
    if prev_quarter==0:
        prev_year=prev_year-1
        prev_quarter=4
    return str(prev_year)+"q"+str(prev_quarter)
    
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    start,bottom = get_recession_start(),get_recession_bottom()
    housing = convert_housing_data_to_quarters()[[one_quarter_before(start),bottom]]
    housing["PriceRatio"] = housing[one_quarter_before(start)]/housing[bottom]
    housing=housing.reset_index()[["State","RegionName","PriceRatio"]].set_index(["State","RegionName"])
    university_towns = get_list_of_university_towns()

    university_towns=university_towns.set_index(["State","RegionName"])
    univ_housing=pd.merge(housing,university_towns,how="inner",left_index=True, right_index=True)
#This is where I don't get pandas syntax and it's throwing errors and I go to the root myself to be verbose
#Not very pandorable or performant and no use of categories and multi level indexes
#Then I manually verify it's correct because it may not be robust as its not easily readable
    n_u_h=housing.reset_index().as_matrix().tolist()
    u_h=univ_housing.reset_index().as_matrix().tolist()
    non_univ_housing=[]

    for row in n_u_h:
        flag=True
        for row2 in u_h:
            if row[0]==row2[0] and row[1]==row2[1]:
                flag=False
        if flag:
            non_univ_housing.append(row)
#Cleanup
    non_univ_housing=pd.DataFrame(non_univ_housing)
    non_univ_housing.columns=["State","RegionName","PriceRatio"]
    non_univ_housing=non_univ_housing.set_index(["State","RegionName"])
#End of the non pandorable part
    result=stats.ttest_ind(univ_housing["PriceRatio"],non_univ_housing["PriceRatio"],nan_policy="omit")
    
    different=False
    if result.pvalue < 0.01 and (result.statistic > 2 or result.statistic < 2): 
        different=True
    better="non-university town"
    if univ_housing["PriceRatio"].mean() < non_univ_housing["PriceRatio"].mean():
        better = "university town"
    return (different,result.pvalue,better)

run_ttest()

/Users/soodd/anaconda/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:24: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


(True, 0.0027240637047531249, 'university town')